In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns# data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import plotly
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Let's load the data in using pandas

In [2]:
df = pd.read_csv('../input/aug-hiring/aug_hiring.csv')

Check the first five rows

In [4]:
df.head()

check the last five rows

In [5]:
df.tail()

check for missing values

In [6]:
df.isnull().sum()

check the number of rows and columns

In [7]:
df.shape

Replace text with numbers

In [8]:
nums = {"experience":{"two": 2,"three":3,"eleven":11,"seven":7,"ten":10,"five":5,'one':1,'four':4,'twelve':12}}

In [9]:
df = df.replace(nums)

Data Visualization

In [24]:
plt.figure(figsize=(20,20))
plt.subplot(2, 2, 1)
sns.histplot(y=df['salary($)'],kde=True,color='red')
plt.ylabel('Salary',fontsize=30)
plt.subplot(2, 2, 2)
plt.scatter(x=df['test_score(out of 10)'],y=df['salary($)'],s=300,color='green')
plt.xlabel('Test Score',fontsize=30)
plt.ylabel('Salary',fontsize=30)
plt.subplot(2, 2, 3)
plt.scatter(x=df['interview_score(out of 10)'],y=df['salary($)'],s=300,color='purple')
plt.xlabel('Interview Score',fontsize=30)
plt.ylabel('Salary',fontsize=30)
plt.subplot(2, 2, 4)
plt.scatter(x=df['experience'],y=df['salary($)'],s=300,color='cyan')
plt.xlabel('Experience',fontsize=30)
plt.ylabel('Salary',fontsize=30)

plt.show()


Let's get the dependent and independent variables

In [11]:
x = df.drop('salary($)',axis=1)
y = df['salary($)']

Split data into train and test sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=2)

Standardize the training sets

In [13]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

Standardize the test sets

In [14]:
scaler=StandardScaler()
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

Create machine learning pipeline

In [15]:
# Create a pipeline
pipe = Pipeline([("regressor", RandomForestRegressor())])
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                {"regressor": [Lasso()],
                 "regressor__alpha":[0.1,0.2]},
                {"regressor": [DecisionTreeRegressor()],
                 "regressor__max_depth":[5,8,15,25,30,None],
                 "regressor__min_samples_leaf":[1,2,5,10,15,100],
                 "regressor__max_leaf_nodes": [2, 5,10] ##This solvers don't allow L1 penalty
                 },
                {"regressor": [RandomForestRegressor()],
                 "regressor__n_estimators": [10, 100, 1000],
                 "regressor__max_depth":[5,8,15,25,30,None],
                 "regressor__min_samples_leaf":[1,2,5,10,15,100],
                 "regressor__max_leaf_nodes": [2, 5,10]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5, verbose=0,n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

check for the best model and parameters

In [16]:
print(best_model.best_estimator_)

Create and predict with the  best model

In [52]:
pipeline_DT = Pipeline([("regressor",DecisionTreeRegressor(max_depth=5, max_leaf_nodes=10))])

In [53]:
salary_model = pipeline_DT.fit(X_train_scaled,y_train)
y_pred = salary_model.predict(X_test)

Evaluate the model

In [54]:
print('mean absolute error:',mean_absolute_error(y_test, y_pred))

In [55]:
print('root mean squared error:', mean_squared_error(y_test, y_pred)**0.5)

Save the model as pickle file

In [56]:
import pickle
file = open('salary_model_2.pkl', 'wb')

pickle.dump(salary_model,file)